In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [17]:
df = pd.read_csv('train.csv')

train = df
print(train.columns)
train.drop(['PassengerId','Ticket','Name','Fare','Cabin'],axis=1,inplace=True)

# 성별 인코딩 남 : 0 여 : 1
train['Sex'] = train['Sex'].map({'male':0,'female':1})

train['Family'] = train['SibSp'] + train['Parch']

train.drop(['SibSp','Parch'],axis=1,
           inplace=True)
#df.info()
train['Embarked'] = train['Embarked'].fillna('Q')
train['Embarked'] = train['Embarked'].map({'S':0, 'C':1, 'Q': 2})
train['Age'] = train['Age'].fillna(train['Age'].mean())

# 나이에 대해서는....Binning 처리를 해요!
train.loc[train['Age'] < 8, 'Age'] = 0
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65), 'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3

x_data = train.drop('Survived',axis=1,inplace=False).values
t_data = train['Survived'].values.reshape(-1,1)

#정규화

scaler = MinMaxScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)



Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [18]:
#keras model

keras_model = Sequential()
keras_model.add(Flatten(input_shape = (5,)))
keras_model.add(Dense(units = 1, activation = 'sigmoid'))
keras_model.compile(optimizer=Adam(learning_rate=1e-2),
                    loss = 'binary_crossentropy',
                    metrics = ['acc'])
keras_model.fit(x_data_norm,
                t_data,
                epochs = 300,
                verbose = 1,
                validation_split = 0.2)
#loss: 0.4536 - acc: 0.8006 - val_loss: 0.3909 - val_acc: 0.8212


Epoch 1/300
23/23 [==============================] - 1s 11ms/step - loss: 0.5841 - acc: 0.7430 - val_loss: 0.5381 - val_acc: 0.8045
Epoch 2/300
23/23 [==============================] - 0s 5ms/step - loss: 0.5563 - acc: 0.7725 - val_loss: 0.5134 - val_acc: 0.8268
Epoch 3/300
23/23 [==============================] - 0s 3ms/step - loss: 0.5350 - acc: 0.7725 - val_loss: 0.4931 - val_acc: 0.8212
Epoch 4/300
23/23 [==============================] - 0s 3ms/step - loss: 0.5185 - acc: 0.7739 - val_loss: 0.4761 - val_acc: 0.8268
Epoch 5/300
23/23 [==============================] - 0s 3ms/step - loss: 0.5061 - acc: 0.7893 - val_loss: 0.4650 - val_acc: 0.8212
Epoch 6/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4965 - acc: 0.7949 - val_loss: 0.4560 - val_acc: 0.8156
Epoch 7/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4889 - acc: 0.7949 - val_loss: 0.4477 - val_acc: 0.8156
Epoch 8/300
23/23 [==============================] - 0s 3ms/step - loss: 0.4824 - 

In [26]:
# 학습이 끝났으니 제출파일 생성
# 모델이 만들어 졌으니 test데이터로 예측된 결과를 csv파일로 만들어서
# 캐글에 제출하고 결과확인
test_df = pd.read_csv('test.csv')
test_df.drop(['PassengerId','Ticket','Name','Fare','Cabin'],axis=1,inplace=True)

# 성별 인코딩 남 : 0 여 : 1
test_df['Sex'] = test_df['Sex'].map({'male':0,'female':1})

test_df['Family'] = test_df['SibSp'] + test_df['Parch']

test_df.drop(['SibSp','Parch'],axis=1,
           inplace=True)
#df.info()
test_df['Embarked'] = test_df['Embarked'].fillna('Q')
test_df['Embarked'] = test_df['Embarked'].map({'S':0, 'C':1, 'Q': 2})
test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())

# 나이에 대해서는....Binning 처리를 해요!
test_df.loc[test_df['Age'] < 8, 'Age'] = 0
test_df.loc[(test_df['Age'] >= 8) & (test_df['Age'] < 20), 'Age'] = 1
test_df.loc[(test_df['Age'] >= 20) & (test_df['Age'] < 65), 'Age'] = 2
test_df.loc[test_df['Age'] >= 65, 'Age'] = 3

x_data_test = test_df.values
scaler.fit(x_data_test)
x_data_test_norm = scaler.transform(x_data_test)
test_predictions = keras_model.predict(x_data_test_norm)

14/14 [==============================] - 0s 690us/step


In [36]:
sub = pd.read_csv('gender_submission.csv')


In [38]:
binary_predictions = (test_predictions > 0.5).astype(int)
sub['Survived'] = binary_predictions
sub

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [39]:
sub.to_csv('my_model_predict.csv', index=False)